In [3]:
import pandas as pd
import unicodedata

# ---------- Hilfsfunktion: Encoding-Reparatur ----------
def fix_mojibake_text(s):
    """
    Repariert häufige UTF-8/CP1252-Mojibake (z.B. 'ErschlieÃŸung', 'prekaÌˆrer')
    und normalisiert anschließend zu NFC (z.B. 'ä' -> 'ä').
    """
    if not isinstance(s, str):
        return s
    # Versuch: als CP1252 Bytes interpretieren und korrekt als UTF-8 dekodieren
    try:
        s_fixed = s.encode("cp1252").decode("utf-8")
    except Exception:
        s_fixed = s
    # Unicode-Normalisierung (z.B. a +  ̈ -> ä)
    s_fixed = unicodedata.normalize("NFC", s_fixed)
    return s_fixed

def fix_mojibake_df(df: pd.DataFrame) -> pd.DataFrame:
    return df.applymap(fix_mojibake_text)

# Schlagwort-Datei laden (mit Spalten: Schlagwort, Gewicht, Systematik, Alleinstehend)
schlagworte_df = pd.read_excel("schlagworte_gewichtung.xlsx")
schlagworte_df.columns = schlagworte_df.columns.str.strip()



# Dictionaries für schnelleren Zugriff
weights = dict(zip(schlagworte_df["Schlagwort"], schlagworte_df["Gewichtung"]))
systematics = dict(zip(schlagworte_df["Schlagwort"], schlagworte_df["Systematik"]))


# O-Sätze laden
saetze_df = pd.read_excel("o.xlsx")
saetze_df.columns = saetze_df.columns.str.strip()
# Encoding-Fix auf die O-Sätze anwenden
saetze_df = fix_mojibake_df(saetze_df)

# ---------- DDC-Blacklist ----------
blacklist_ddc = [
    "360", "590", "910", "333", "355", "370",
    "510", "610", "621", "690", "796", "797", "798", "799", "914", "K"
]
if "DDC" in saetze_df.columns:
    saetze_df = saetze_df[~saetze_df["DDC"].astype(str).str.startswith(tuple(blacklist_ddc), na=False)]

# Alle Spalten außer "IDN" durchsuchen
spalten_zum_pruefen = [col for col in saetze_df.columns if col != "IDN"]

def evaluate_row(row):
    text_all = " ".join(str(val) for val in row if not pd.isna(val)).lower()
    word_counts = {}

    for word in weights.keys():
        count = text_all.count(word.lower())
        if count > 0:
            gewicht = weights.get(word, 1)
            score = count * gewicht
            word_counts[word] = (count, score, systematics.get(word, ""))

    if not word_counts:
        return 0, "", 0, ""

    total_weight = sum(weights.get(w, 1) for w in word_counts.keys())

    # Entscheidungskriterien
    if len(word_counts) >= 3 or total_weight >= 3:
        formatted = [
            f"{word} ({count} Treffer, Gewicht={weights.get(word,1)}, Score={score}, Systematik={syst})"
            for word, (count, score, syst) in word_counts.items()
        ]
        return len(word_counts), "; ".join(formatted), total_weight, "OK"



# Neue Spalten erzeugen
saetze_df[["Count", "Gefundene_Schlagworte", "Gesamtgewichtung", "Status"]] = saetze_df[spalten_zum_pruefen].apply(
    lambda row: pd.Series(evaluate_row(row)), axis=1
)

# Nur Zeilen mit Status=OK übernehmen
ergebnis_df = saetze_df[saetze_df["Status"] == "OK"][["IDN", "Titel","Zusatztitel", "DDC", "Count", "Gesamtgewichtung", "Gefundene_Schlagworte"]]

# --- Zeilen mit "Roman" im Titel oder Zusatztitel löschen ---
for col in ["Titel", "Zusatztitel"]:
    if col in ergebnis_df.columns:
        ergebnis_df = ergebnis_df[~ergebnis_df[col].str.contains(r"\broman\b", case=False, na=False)]


# Ergebnis speichern
ergebnis_df.to_excel("neu_gefilterte_saetze.xlsx", index=False)

print("Fertig! Datei 'gefilterte_saetze.xlsx' erstellt ✅")


Fertig! Datei 'gefilterte_saetze.xlsx' erstellt ✅
